<a href="https://colab.research.google.com/github/ndsoi/DrawPixel/blob/main/Beyes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 第一部分：实现一个简易的朴素贝叶斯分类器


# 第一部分：实现一个简易的朴素贝叶斯分类器
主要内容包括<br>
1. 划分训练集和测试集
2. 计算P(c)的函数
3. 计算P(x_i|c)


## 原始数据

In [1]:
dataSet = [
        # 1
        ['青绿', '蜷缩', '浊响', '清晰', '凹陷', '硬滑', '好瓜'],
        # 2
        ['乌黑', '蜷缩', '沉闷', '清晰', '凹陷', '硬滑', '好瓜'],
        # 3
        ['乌黑', '蜷缩', '浊响', '清晰', '凹陷', '硬滑', '好瓜'],
        # 4
        ['青绿', '蜷缩', '沉闷', '清晰', '凹陷', '硬滑', '好瓜'],
        # 5
        ['浅白', '蜷缩', '浊响', '清晰', '凹陷', '硬滑', '好瓜'],
        # 6
        ['青绿', '稍蜷', '浊响', '清晰', '稍凹', '软粘', '好瓜'],
        # 7
        ['乌黑', '稍蜷', '浊响', '稍糊', '稍凹', '软粘', '好瓜'],
        # 8
        ['乌黑', '稍蜷', '浊响', '清晰', '稍凹', '硬滑', '好瓜'],

        # ----------------------------------------------------
        # 9
        ['乌黑', '稍蜷', '沉闷', '稍糊', '稍凹', '硬滑', '坏瓜'],
        # 10
        ['青绿', '硬挺', '清脆', '清晰', '平坦', '软粘', '坏瓜'],
        # 11
        ['浅白', '硬挺', '清脆', '模糊', '平坦', '硬滑', '坏瓜'],
        # 12
        ['浅白', '蜷缩', '浊响', '模糊', '平坦', '软粘', '坏瓜'],
        # 13
        ['青绿', '稍蜷', '浊响', '稍糊', '凹陷', '硬滑', '坏瓜'],
        # 14
        ['浅白', '稍蜷', '沉闷', '稍糊', '凹陷', '硬滑', '坏瓜'],
        # 15
        ['乌黑', '稍蜷', '浊响', '清晰', '稍凹', '软粘', '坏瓜'],
        # 16
        ['浅白', '蜷缩', '浊响', '模糊', '平坦', '硬滑', '坏瓜'],
        # 17
        ['青绿', '蜷缩', '沉闷', '稍糊', '稍凹', '硬滑', '坏瓜']
    ]
Attr = ['色泽', '根蒂', '敲击', '纹理', '脐部', '触感','类别']

# 将数据合并格式
D = []
for i in range(len(dataSet)):
  d = {}
  for j in range(len(Attr)):
    d[Attr[j]] = dataSet[i][j]
  D.append(d)

## 划分训练集和测试集

In [2]:
# 辅助函数：用于单行打印数据项
def show(data):
  for i in data:
    print(i)

In [3]:
import numpy as np

# 这里其实已经定了Pc了
good_melon_nums = 5  # 抽取5个好瓜
bad_melon_nums = 5  # 抽取5个坏瓜


def divide_train_and_test(original_data):

  good_index = np.arange(0, 8, 1, dtype=np.int16)
  bad_index = np.arange(0,9,1,dtype=np.int16)
  # 打乱
  np.random.shuffle(good_index)
  np.random.shuffle(bad_index)

  print(f'好瓜的索引是:{good_index}')
  print(f'坏瓜的索引是:{bad_index}')
  # 按index中的索引,取good_melon_nums个好瓜和bad_melon_nums个坏瓜作为训练集
  train_data = [ original_data[i] for i in good_index[:good_melon_nums] ] + [ original_data[i+8] for i in bad_index[:bad_melon_nums] ]
  # 将剩余索引所代表的瓜分给测试集
  test_data = [ original_data[i] for i in good_index[good_melon_nums:]] + [ original_data[i+8] for i in bad_index[bad_melon_nums:] ]

  return train_data,test_data

train_data,test_data = divide_train_and_test(D)
show(train_data)
show(test_data)

好瓜的索引是:[0 5 7 2 4 6 3 1]
坏瓜的索引是:[5 1 2 8 7 4 3 0 6]
{'色泽': '青绿', '根蒂': '蜷缩', '敲击': '浊响', '纹理': '清晰', '脐部': '凹陷', '触感': '硬滑', '类别': '好瓜'}
{'色泽': '青绿', '根蒂': '稍蜷', '敲击': '浊响', '纹理': '清晰', '脐部': '稍凹', '触感': '软粘', '类别': '好瓜'}
{'色泽': '乌黑', '根蒂': '稍蜷', '敲击': '浊响', '纹理': '清晰', '脐部': '稍凹', '触感': '硬滑', '类别': '好瓜'}
{'色泽': '乌黑', '根蒂': '蜷缩', '敲击': '浊响', '纹理': '清晰', '脐部': '凹陷', '触感': '硬滑', '类别': '好瓜'}
{'色泽': '浅白', '根蒂': '蜷缩', '敲击': '浊响', '纹理': '清晰', '脐部': '凹陷', '触感': '硬滑', '类别': '好瓜'}
{'色泽': '浅白', '根蒂': '稍蜷', '敲击': '沉闷', '纹理': '稍糊', '脐部': '凹陷', '触感': '硬滑', '类别': '坏瓜'}
{'色泽': '青绿', '根蒂': '硬挺', '敲击': '清脆', '纹理': '清晰', '脐部': '平坦', '触感': '软粘', '类别': '坏瓜'}
{'色泽': '浅白', '根蒂': '硬挺', '敲击': '清脆', '纹理': '模糊', '脐部': '平坦', '触感': '硬滑', '类别': '坏瓜'}
{'色泽': '青绿', '根蒂': '蜷缩', '敲击': '沉闷', '纹理': '稍糊', '脐部': '稍凹', '触感': '硬滑', '类别': '坏瓜'}
{'色泽': '浅白', '根蒂': '蜷缩', '敲击': '浊响', '纹理': '模糊', '脐部': '平坦', '触感': '硬滑', '类别': '坏瓜'}
{'色泽': '乌黑', '根蒂': '稍蜷', '敲击': '浊响', '纹理': '稍糊', '脐部': '稍凹', '触感': '软粘', '类别': '好瓜'}
{'色泽': '青绿', 

## 计算 P(xi|c)


1. 离散属性
$$p(x_i|c) = \frac{|D_{c,x_i}|}{D_c}$$

In [10]:
# 离散属性的
# 获取类别为c的数据列，用于计算多个属性的P(xi|c)
def getDc(train_data,Class):
  Dc = []
  for melon in train_data:
    if melon['类别'] == Class:
      Dc.append(melon)
  return Dc

# 计算类别c中属性attr取值为value的样本概率
def calP_xiForI(attr,value,Dc):
  cnt = 0
  for melon in Dc:
    if melon[attr] == value:
      cnt+=1
  #print(f"P(({attr}={value})|{melon['类别']})={cnt/len(Dc)}")
  return cnt/len(Dc)



2、连续属性
$$p(x_i|c) = \frac{1}{\sqrt{2\pi\sigma_{c,i}}}e^{(-\frac{(x-\mu_{c,i})^2}{2\sigma^2_{c,i}})}$$
对连续属性可考虑概率密度函数，假定p(xi|c)～N(μc,i)，其中
$$\mu_{c,i} = \frac{1}{|D_{c,i}|}\sum_{x_i\epsilon D_c} x_i$$和$$σ_{c,i}^2 = \frac{1}{|D_{c,i}|} \sum_{x_i\epsilon D_{c,i}}(x_i-\mu _c)(x_i-\mu _c)^T$$分别是第c类样本在第i个属性上取值的均值和方差


In [ ]:
import math
### 计算均值
def calmu_c_i(Dc,attr):
  mu = 0
  for data in Dc:
    mu += data[attr]
  return mu/len(Dc)

### 计算方差
def cal_sigma_c_i(Dc,attr):
  sigma = 0
  mu = calmu_c_i(Dc,attr)
  for data in Dc:
    sigma+=(data[attr]-mu)*(data[attr]-mu)
  return sigma/len(Dc)

def calP_xiForC(attr,Dc,test):
  sigma = cal_sigma_c_i(Dc,attr)
  mu = calmu_c_i(Dc,attr)
  tmp = -(test[attr]-mu)*(test[attr]-mu)/(2*sigma*sigma)
  return 1/math.sqrt(2*math.pi*sigma)*math.exp(tmp)


## 计算P(c)
计算P(xi|c)的时候，求出了Dc,则$$p(x_i|c)=\frac{|Dc|}{|D|}$$

## 计算P(c|x)

In [11]:
def calPcx(train_data,Class,test,Attrs):
  # 找出类别为Class的数据
  Dc = getDc(train_data,Class)
  Pc = len(Dc)/len(train_data)
 # print(f"P({Class})={Pc}")
  ans = 1
  for attr,value in test.items():
    # 属性是离散的
    if Attrs[attr] == 'i':
      tmp = calP_xiForI(attr,value,Dc)
    else:
      tmp = calP_xiForC(attr,Dc,test)
    # 属性是连续的
    ans*= tmp

  return ans*Pc


## 分类样本
根据P(c|x)大小判断样本x的类别

In [12]:
# 西瓜这里仅有两类别——好瓜/坏瓜
def classify(train_data,test):
  P_c1 = ['好瓜',calPcx(train_data,'好瓜',test)]
  P_c2 = ['坏瓜',calPcx(train_data,'坏瓜',test)]
  if P_c1[1]<P_c2[1]:
    return P_c2
  else:
    return P_c1
def predict(train_data,test_data):
  res = []
  for i in test_data:
    res.append([i,classify(train_data,i)])
  return res


res = predict(train_data,test_data)
show(res)

P(好瓜)=0.5
P((色泽=乌黑)|好瓜)=0.4
P((根蒂=稍蜷)|好瓜)=0.4
P((敲击=浊响)|好瓜)=1.0
P((纹理=稍糊)|好瓜)=0.0
P((脐部=稍凹)|好瓜)=0.4
P((触感=软粘)|好瓜)=0.2
P((类别=好瓜)|好瓜)=1.0
P(坏瓜)=0.5
P((色泽=乌黑)|坏瓜)=0.0
P((根蒂=稍蜷)|坏瓜)=0.2
P((敲击=浊响)|坏瓜)=0.2
P((纹理=稍糊)|坏瓜)=0.4
P((脐部=稍凹)|坏瓜)=0.2
P((触感=软粘)|坏瓜)=0.2
P((类别=好瓜)|坏瓜)=0.0
P(好瓜)=0.5
P((色泽=青绿)|好瓜)=0.4
P((根蒂=蜷缩)|好瓜)=0.6
P((敲击=沉闷)|好瓜)=0.0
P((纹理=清晰)|好瓜)=1.0
P((脐部=凹陷)|好瓜)=0.6
P((触感=硬滑)|好瓜)=0.8
P((类别=好瓜)|好瓜)=1.0
P(坏瓜)=0.5
P((色泽=青绿)|坏瓜)=0.4
P((根蒂=蜷缩)|坏瓜)=0.4
P((敲击=沉闷)|坏瓜)=0.4
P((纹理=清晰)|坏瓜)=0.2
P((脐部=凹陷)|坏瓜)=0.2
P((触感=硬滑)|坏瓜)=0.8
P((类别=好瓜)|坏瓜)=0.0
P(好瓜)=0.5
P((色泽=乌黑)|好瓜)=0.4
P((根蒂=蜷缩)|好瓜)=0.6
P((敲击=沉闷)|好瓜)=0.0
P((纹理=清晰)|好瓜)=1.0
P((脐部=凹陷)|好瓜)=0.6
P((触感=硬滑)|好瓜)=0.8
P((类别=好瓜)|好瓜)=1.0
P(坏瓜)=0.5
P((色泽=乌黑)|坏瓜)=0.0
P((根蒂=蜷缩)|坏瓜)=0.4
P((敲击=沉闷)|坏瓜)=0.4
P((纹理=清晰)|坏瓜)=0.2
P((脐部=凹陷)|坏瓜)=0.2
P((触感=硬滑)|坏瓜)=0.8
P((类别=好瓜)|坏瓜)=0.0
P(好瓜)=0.5
P((色泽=青绿)|好瓜)=0.4
P((根蒂=稍蜷)|好瓜)=0.4
P((敲击=浊响)|好瓜)=1.0
P((纹理=稍糊)|好瓜)=0.0
P((脐部=凹陷)|好瓜)=0.6
P((触感=硬滑)|好瓜)=0.8
P((类别=坏瓜)|好瓜)=0.0
P(坏瓜)=0.5
P((色泽=青绿)|坏瓜)=0.4
P((根蒂=稍蜷)|坏瓜)=0.2
P(

# 第二部分:拉普拉斯修正
上述训练存在的问题是：<br>
由于多属性之间假设为相互独立，概率值是连乘的形式，当有一个属性出现的概率为0的时候，会将其他属性的信息值“抹去”<br>
修正方式是:<br>
1. $$P(c) = \frac{|Dc|+1}{|D|+N}$$
2. $$P(c,i) = \frac{|D_{c,i}|+1}{|D_c|+N_i}$$
令N表示训练集D中可能的类别数，Ni表示第i个属性可能的取值数



## 计算 P(xi|c)


In [18]:
def getNi(train_data,attr):
  N_dict = {}
  for data in train_data:
    N_dict[data[attr]] = 1

  return len(N_dict.keys())


# 计算类别c中属性attr取值为value的样本概率
def calP_xiForI_v2(attr,value,Dc):
  cnt = 0
  for melon in Dc:
    if melon[attr] == value:
      cnt+=1
  #print(f"P(({attr}={value})|{melon['类别']})={cnt/len(Dc)}")
  return (cnt+1)/(len(Dc)+getNi(train_data,attr))



## 计算P(c|x)

In [16]:
# 类别的种类数
N = 2
def calPcx_v2(train_data,Class,test):
  # 找出类别为Class的数据
  Dc = getDc(train_data,Class)
  Pc = (len(Dc)+1)/(len(train_data)+N)
 # print(f"P({Class})={Pc}")
  ans = 1
  for attr,value in test.items():
    tmp = calP_xiForI_v2(attr,value,Dc)
    ans*= tmp

  return ans*Pc


In [19]:
# 西瓜这里仅有两类别——好瓜/坏瓜
def classify_v2(train_data,test):
  P_c1 = ['好瓜',calPcx_v2(train_data,'好瓜',test)]
  P_c2 = ['坏瓜',calPcx_v2(train_data,'坏瓜',test)]
  if P_c1[1]<P_c2[1]:
    return P_c2
  else:
    return P_c1
def predict_v2(train_data,test_data):
  res = []
  for i in test_data:
    res.append([i,classify_v2(train_data,i)])
  return res


res = predict_v2(train_data,test_data)
show(res)

# 相较未修正之前，正确率提高了50%

[{'色泽': '乌黑', '根蒂': '稍蜷', '敲击': '浊响', '纹理': '稍糊', '脐部': '稍凹', '触感': '软粘', '类别': '好瓜'}, ['好瓜', 0.0006053691007653061]]
[{'色泽': '青绿', '根蒂': '蜷缩', '敲击': '沉闷', '纹理': '清晰', '脐部': '凹陷', '触感': '硬滑', '类别': '好瓜'}, ['好瓜', 0.002690529336734694]]
[{'色泽': '乌黑', '根蒂': '蜷缩', '敲击': '沉闷', '纹理': '清晰', '脐部': '凹陷', '触感': '硬滑', '类别': '好瓜'}, ['好瓜', 0.002690529336734694]]
[{'色泽': '青绿', '根蒂': '稍蜷', '敲击': '浊响', '纹理': '稍糊', '脐部': '凹陷', '触感': '硬滑', '类别': '坏瓜'}, ['坏瓜', 0.0006726323341836735]]
[{'色泽': '浅白', '根蒂': '蜷缩', '敲击': '浊响', '纹理': '模糊', '脐部': '平坦', '触感': '软粘', '类别': '坏瓜'}, ['坏瓜', 0.0010762117346938774]]
[{'色泽': '乌黑', '根蒂': '稍蜷', '敲击': '沉闷', '纹理': '稍糊', '脐部': '稍凹', '触感': '硬滑', '类别': '坏瓜'}, ['坏瓜', 0.00033631616709183676]]
[{'色泽': '乌黑', '根蒂': '稍蜷', '敲击': '浊响', '纹理': '清晰', '脐部': '稍凹', '触感': '软粘', '类别': '坏瓜'}, ['好瓜', 0.0006053691007653061]]
